In [ ]:
import glob
import os
import warnings
import mlflow
import mlflow.pytorch 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split
from torchvision import transforms
from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [ ]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())
    ### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [ ]:
# loading data
file_names = ["train_df_3_cat.csv", "test_df_3_cat.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Unpack the dataframes into separate variables
train_df, test_df = dataframes

print("DataFrames have been loaded successfully.")


In [ ]:
# Defining small sample vs. end smaple
whole_data_set = False
# end sample or small sample    
if whole_data_set == True:
    print("Using the whole data set")
else:

    test_df = test_df.sample(n=20, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a small sample : 48692</span>'))

Calss definition dataloader (do not change over models)

Step 1: U-Net Model and Data Preparation

predictions

In [ ]:
train_df

test predicion

In [ ]:

test_df = test_df.drop(['severity', 'condition', 'level', 'series_id', 'missing_image'], axis=1)


In [ ]:
print(test_df.to_string(index=False, header=True)) 

predicting

In [ ]:
test_data = test_df

In [ ]:
# Replace <run_id> with your actual run ID and <path_to_model> with the artifact path used during logging
model_path = r"C:\Users\HP1\Desktop\Spiced\capstone-project\mlruns\388846461532362513\3f95f17d16ec4e53bb1ef451c608886e\artifacts\final_model"
model_for_prediciton = mlflow.pytorch.load_model(model_path)

model_for_prediciton.eval()  # Set the model to evaluation mode

In [ ]:
s

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
import mlflow
import mlflow.pytorch
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from PIL import Image
import pydicom

class MRILocalizationDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['image_path']
        dicom_image = pydicom.dcmread(img_path)
        image_array = dicom_image.pixel_array
        image = Image.fromarray(image_array)

        if image.mode != 'RGB':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image  # No coordinates returned, as we are predicting them

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Set the model to evaluation mode
model_for_prediction.eval()

# Prepare the new test dataset (without coordinates) and DataLoader
test_dataset = MRILocalizationDataset(data=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_for_prediction.to(device)

# Initialize the list to store predicted coordinates
predicted_coords = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model_for_prediction(images)  # Predict x, y coordinates
        outputs = outputs.cpu().numpy()
# Check the output shape directly
print(f"Output shape: {outputs.shape}")    

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
import mlflow
import mlflow.pytorch
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from PIL import Image
import pydicom

class MRILocalizationDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['image_path']
        dicom_image = pydicom.dcmread(img_path)
        image_array = dicom_image.pixel_array
        image = Image.fromarray(image_array)

        if image.mode != 'RGB':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image  # No coordinates returned, as we are predicting them

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Set the model to evaluation mode
model_for_prediction.eval()

# Prepare the new test dataset (without coordinates) and DataLoader
test_dataset = MRILocalizationDataset(data=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_for_prediction.to(device)

# Initialize the list to store predicted coordinates
predicted_coords = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model_for_prediction(images)  # Predict x, y coordinates
        outputs = outputs.cpu().numpy()

        for output in outputs:
            x_pred, y_pred = output  # Predicted coordinates for x, y
            
            # Apply scaling if images were resized during training
            scaling_factor = 256 / 128  # Adjust based on your resizing factors
            x_original = x_pred * scaling_factor
            y_original = y_pred * scaling_factor

            predicted_coords.append((x_original, y_original))

# Save the predicted coordinates along with the original test data
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])
test_df_with_preds = pd.concat([test_df.reset_index(drop=True), output_df], axis=1)
test_df_with_preds.to_csv("test_with_predicted_coordinates.csv", index=False)
 

In [ ]:
output_df  


In [ ]:
test_df_with_preds

In [ ]:


predicted_coords = []  # Initialize the list to store predicted coordinates

with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        heatmaps = model(images)
        heatmaps = heatmaps.cpu().numpy()

        # Check if heatmaps is 4D (batch_size, channels, height, width)
        if heatmaps.ndim == 4:
            for batch_idx in range(heatmaps.shape[0]):  # Iterate over batch
                heatmap = heatmaps[batch_idx, 0]  # Assuming we have only one channel
                heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap)) if np.max(heatmap) != np.min(heatmap) else heatmap

                if np.max(heatmap) > 0.5:  # Adjust the threshold as needed
                    y, x = np.unravel_index(np.argmax(heatmap, axis=None), heatmap.shape)
                    predicted_coords.append((x, y))
                else:
                    predicted_coords.append((np.nan, np.nan))  # Assign NaN if no valid peak
        else:
            print(f"Unexpected heatmap dimensions: {heatmaps.shape}")

# Save the predicted coordinates along with the original test data
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])
test_df_with_preds = pd.concat([test_df.reset_index(drop=True), output_df], axis=1)
test_df_with_preds.to_csv("test_with_predicted_coordinates.csv", index=False)

print("Inference complete. Predicted coordinates saved.")
 

In [ ]:
test_df_with_preds

In [ ]:
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        heatmaps = model(images)
        heatmaps = heatmaps.cpu().numpy()

        for heatmap in heatmaps:
            print(f"Heatmap shape: {heatmap.shape}")
            print(f"Heatmap values: {np.min(heatmap)}, {np.max(heatmap)}")
            if heatmap.ndim == 2:
                y, x = np.unravel_index(np.argmax(heatmap, axis=None), heatmap.shape)
                print(f"Predicted coordinates: x={x}, y={y}")
                predicted_coords.append((x, y))


In [ ]:
test_df_with_preds

In [ ]:
import numpy as np
import pandas as pd
import torch
import mlflow
from torch.utils.data import DataLoader

# Load the trained model from MLflow
model_path = "C:/Users/HP1/Desktop/Spiced/capstone-project/mlruns/491281383988954356/344ed7741b8b4cb9bf87844ab1e511af/artifacts/final_model"
model = mlflow.pytorch.load_model(model_path)
model.eval()  # Set the model to evaluation mode

# Prepare the new test dataset (without coordinates) and DataLoader
test_dataset = MRILocalizationDataset(data=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Ensure the model is on the right device

# Inference on new test data
predicted_coords = []
with torch.no_grad():  # Disable gradient computation for inference
    for images, _ in test_loader:  # Unpack images and ignore labels
        images = images.to(device)
        outputs = model(images)  # Get model output
        outputs = outputs.view(outputs.size(0), -1)  # Reshape to (batch_size, 2) for x, y coordinates
        predicted_coords.extend(outputs.cpu().numpy())  # Move to CPU and add to results

# Convert the predicted coordinates to a DataFrame
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])

# Combine original test_df with predicted coordinates
test_df_with_preds = pd.concat([test_df.reset_index(drop=True), output_df], axis=1)



In [ ]:
test_df_with_preds